In [1]:
import pandas as pd
import os
import glob
from datetime import datetime

In [2]:
path = os.getcwd() + "\\Dataset\\"
files = glob.glob(path + "/*precios*")

In [96]:
df_precios = pd.read_excel(files[0], dtype=object ,sheet_name=1)
df_precios1 = pd.read_excel(files[0], sheet_name=0)
df_precios2 = pd.read_csv(files[1],sep=',',encoding='utf-16')
df_precios3 = pd.read_json(files[2])

In [175]:
df_precios1 = pd.read_excel(files[0], sheet_name=0)

In [230]:
def unDate(x):
    """read_excel me trae algunas sucursal_id como si fueran datetime. Esta funcion toma elementos, chequea si son datetime y en caso de serlo, los convierte a str con el formato acorde al orden de identificadores de sucursal"""
    if isinstance(x,datetime):
        x=x.strftime("%d-%m-%Y")
    return x

def nasDups(df):
    """Cosas básicas: remover duplicados y nulos"""
    df = df.copy()
    df = df.dropna()
    df = df.drop_duplicates()
    return df

def typeCheck(df):
    """En base a la exploracion inicial, se detectaron errores comunes en los diferentes campos. Esta funcion soluciona todos los problemas encontrados en los diferentes archivos de la fuente de datos"""

    if df.precio.dtype != "float":
        """Algunos precios llegan con un string vacio en vez de nulo. Los convertimos a None para removerlos despues. A los que tienen valores, los convertimos a float"""
        try:
            df.precio = df.precio.apply(lambda x: None if x == '' else x)
            df.precio = df.precio.astype(float)
        except:
            print("Error type precios")

    if df.sucursal_id.dtype != "str":
        try:
            df.sucursal_id = df.sucursal_id.astype('string')
        except:
            print('Error type sucursal_id')
            
    if df.producto_id.dtype != "string":
        """Primero chequeo si el producto_id es string o no. De no serlo, es uno de los casos donde está en float o notacion cientifica. Primero lo paso a int y luego a string."""
        try:
            df.producto_id = df.producto_id.astype('int64')
            df.producto_id = df.producto_id.astype('string')
        except:
            print("Jaja, no")
        return df

def splitProd(df):
    """Algunos productos tenian hardcodeada la sucursal. Removemos esa porcion del string y nos quedamos unicamente con el codigo de producto (12-13 char)"""
    df.producto_id = df.producto_id.str.split('-').str[-1]
    return df


In [231]:
def etl(df):
    df = nasDups(df)
    df = typeCheck(df)
    df = splitProd(df)
    df = unDate(df)
    return df

In [232]:
df_precios = etl(df_precios)
df_precios1 = etl(df_precios1)
df_precios2 = etl(df_precios2)
df_precios3 = etl(df_precios3)